# PROJET CRYPTOGRAPHY
# 1- Partie Code Source 




# Team Members :
- Mrabet Ameur Hadjer 191931080198
- Bouta Ali
- Ali-khodja Myriam Johanna 
- Abouab Walid
- Benosmane Riad

- # Cette partie est dédiée au chiffrement et déchiffrement RSA en utilisant toutes les optimisations possibles ainsi les bases de l'arithmétique modulaire :

In [1]:
import random, sys, os, math, base64 
from cryptography.fernet import Fernet

# Part 1: Prime numbers and modular inverse

# Computation of Greatest Common Divisor of a and b using Euclid's Algorithm

In [2]:
def gcd(a, b):
    # Return the Greatest Common Divisor of a and b using Euclid's Algorithm
    while a != 0:
        a, b = b % a, a        
    return b

**Question:** Check the Euclid's Algorithm and try different values

In [3]:
#This is an example
gcd(150,3)

3

# Computation of the modular inverse of a % m

In [4]:
def findModInverse(a, m):
    # Return the modular inverse of a % m, which is
    # the number x such that a*x % m = 1

    if gcd(a, m) != 1:
        return None # No mod inverse exists if a & m aren't relatively prime.

    # Calculate using the Extended Euclidean Algorithm:
    u1, u2, u3 = 1, 0, a
    v1, v2, v3 = 0, 1, m
    while v3 != 0:
        q = u3 // v3 # Note that // is the integer division operator
        v1, v2, v3, u1, u2, u3 = (u1 - q * v1), (u2 - q * v2), (u3 - q * v3), v1, v2, v3
    return u1 % m

**Question:** Check the Extended Euclidean Algorithm and try different values

In [5]:
#This is an example
findModInverse(4, 7)

2

# Algorithms to verify if a number is a prime number

In [6]:
def isPrimeTrialDiv(num):
    # Returns True if num is a prime number, otherwise False.

    # Uses the trial division algorithm for testing primality.

    # All numbers less than 2 are not prime:
    if num < 2:
        return False

    # See if num is divisible by any number up to the square root of num:
    for i in range(2, int(math.sqrt(num)) + 1):
        if num % i == 0:
            return False
    return True

**Question:** Check the algorithm and try different values

In [7]:
#This is an example
isPrimeTrialDiv(13)

True

In [8]:
def primeSieve(sieveSize):
    # Returns a list of prime numbers calculated using
    # the Sieve of Eratosthenes algorithm.

    sieve = [True] * sieveSize
    sieve[0] = False # Zero and one are not prime numbers.
    sieve[1] = False

    # Create the sieve:
    for i in range(2, int(math.sqrt(sieveSize)) + 1):
        pointer = i * 2
        while pointer < sieveSize:
            sieve[pointer] = False
            pointer += i

    # Compile the list of primes:
    primes = []
    for i in range(sieveSize):
        if sieve[i] == True:
            primes.append(i)

    return primes

**Question:** Try different values

In [9]:
#This is an example
n = primeSieve(200)
print(n)


[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199]


# Rabin-Miller Algorithm

In [10]:
def rabinMiller(num):
    # Returns True if num is a prime number.
    if num % 2 == 0 or num < 2:
        return False # Rabin-Miller doesn't work on even integers.
    if num == 3:
        return True
    s = num - 1
    t = 0
    while s % 2 == 0:
        # Keep halving s until it is odd (and use t
        # to count how many times we halve s):
        s = s // 2
        t += 1
    for trials in range(5): # Try to falsify num's primality 5 times.
        a = random.randrange(2, num - 1)
        v = pow(a, s, num)
        if v != 1: # (This test does not apply if v is 1.)
            i = 0
            while v != (num - 1):
                if i == t - 1:
                    return False
                else:
                    i = i + 1
                    v = (v ** 2) % num
    return True

**Question:** How works the algorithm? Instrument the algorithm and try different values.

In [11]:
#This is an example
rabinMiller(37)

True

# Optimized prime number verifier algorithm

In [12]:
# Most of the time we can quickly determine if num is not prime
# by dividing by the first few dozen prime numbers. This is quicker
# than rabinMiller(), but does not detect all composites.
LOW_PRIMES = primeSieve(100)
print(LOW_PRIMES)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


In [13]:
def isPrime(num):
    # Return True if num is a prime number. This function does a quicker
    # prime number check before calling rabinMiller().
    if (num < 2):
        return False # 0, 1, and negative numbers are not prime.
    # See if any of the low prime numbers can divide num:
    for prime in LOW_PRIMES:
        if (num % prime == 0):
            return False
        if (num == prime):
            return True
    # If all else fails, call rabinMiller() to determine if num is a prime:
    return rabinMiller(num)

**Question:** Check the algorithm and try different values

In [14]:
# This is an exmaple
isPrime(2048)

False

# Large prime number generation

In [15]:
def generateLargePrime(keysize=1024):
    # Return a random prime number that is keysize bits in size.
    i=0
    while True:
        num = random.randrange(2**(keysize-1), 2**(keysize))
        #print('random number', i, hex(num))
        i = i + 1
        if isPrime(num):
            #print('number of tries:', i)
            return num

**Question:** Explain the algorithm and generate different prime numbers. What can you say?

In [16]:
key = generateLargePrime(24)
print(hex(key))

0x93601f


# Part 2: RSA Algorithm

# RSA public key and private key generation

In [17]:
def generateKey(keySize):
    # Creates a public/private keys keySize bits in size.
    p = 0
    q = 0
    # Step 1: Create two prime numbers, p and q. Calculate n = p * q.
    print('Generating p & q primes...')
    while p == q:
        p = generateLargePrime(keySize)
        q = generateLargePrime(keySize)
    n = p * q

    # Step 2: Create a number e that is relatively prime to (p-1)*(q-1):
    print('Generating e that is relatively prime to (p-1)*(q-1)...')
    while True:
        # Keep trying random numbers for e until one is valid:
        e = random.randrange(2 ** (keySize - 1), 2 ** (keySize))
        if gcd(e, (p - 1) * (q - 1)) == 1:
            break

    # Step 3: Calculate d, the mod inverse of e:
    print('Calculating d that is mod inverse of e...')
    d = findModInverse(e, (p - 1) * (q - 1))

    publicKey = (n, e)
    privateKey = (n, d)

    return (publicKey, privateKey)

**Question:** Explain the algorithm and run it several times.

In [18]:
# This is an example
publicKey, privateKey = generateKey(128)
print('Public key:', publicKey)
print('Private key:', privateKey)

Generating p & q primes...
Generating e that is relatively prime to (p-1)*(q-1)...
Calculating d that is mod inverse of e...
Public key: (77942802977250303913701122038036502620833098300622821145749640561728505309767, 218882845325691719870993322518699622667)
Private key: (77942802977250303913701122038036502620833098300622821145749640561728505309767, 17974356483684588619698046000643647812347129502184819644081467579178235300603)


# Storing public and private keys into files

In [19]:
def makeKeyFiles(name, keySize):
    # Creates two files 'x_pubkey.txt' and 'x_privkey.txt' (where x
    # is the value in name) with the n,e and d,e integers written in
    # them, delimited by a comma.

    # Our safety check will prevent us from overwriting our old key files:
    if os.path.exists('%s_pubkey.txt' % (name)) or os.path.exists('%s_privkey.txt' % (name)):
        sys.exit('WARNING: The file %s_pubkey.txt or %s_privkey.txt already exists! Use a different name or delete these files and re-run this program.' % (name, name))

    publicKey, privateKey = generateKey(keySize)

    print()
    print('The public key is a %s and a %s digit number.' % (len(str(publicKey[0])), len(str(publicKey[1]))))
    print('Writing public key to file %s_pubkey.txt...' % (name))
    fo = open('%s_pubkey.txt' % (name), 'w')
    fo.write('%s,%s,%s' % (keySize, publicKey[0], publicKey[1]))
    fo.close()

    print()
    print('The private key is a %s and a %s digit number.' % (len(str(publicKey[0])), len(str(publicKey[1]))))
    print('Writing private key to file %s_privkey.txt...' % (name))
    fo = open('%s_privkey.txt' % (name), 'w')
    fo.write('%s,%s,%s' % (keySize, privateKey[0], privateKey[1]))
    fo.close()

**Question:** Explain the **makeKeyFiles** function and verify files have been generated

In [21]:
# Create a public/private keypair with 1024 bit keys:
print('Making key files...')
makeKeyFiles('your_file', 1024)
print('Key files made.')


Making key files...
Generating p & q primes...
Generating e that is relatively prime to (p-1)*(q-1)...
Calculating d that is mod inverse of e...

The public key is a 617 and a 308 digit number.
Writing public key to file your_file_pubkey.txt...

The private key is a 617 and a 308 digit number.
Writing private key to file your_file_privkey.txt...
Key files made.


# RSA encryption

In [22]:
# IMPORTANT: The block size MUST be less than or equal to the key size!
# (Note: The block size is in bytes, the key size is in bits. There
# are 8 bits in 1 byte.)

DEFAULT_BLOCK_SIZE = 128 # 128 bytes
BYTE_SIZE = 256 # One byte has 256 different values.

def getBlocksFromText(message, blockSize=DEFAULT_BLOCK_SIZE):
    # Converts a string message to a list of block integers. Each integer
    # represents 128 (or whatever blockSize is set to) string characters.

    messageBytes = message.encode('ascii') # convert the string to bytes

    blockInts = []
    for blockStart in range(0, len(messageBytes), blockSize):
        # Calculate the block integer for this block of text
        blockInt = 0
        for i in range(blockStart, min(blockStart + blockSize, len(messageBytes))):
            blockInt += messageBytes[i] * (BYTE_SIZE ** (i % blockSize))
        blockInts.append(blockInt)
    return blockInts


def encryptMessage(message, key, blockSize=DEFAULT_BLOCK_SIZE):
    # Converts the message string into a list of block integers, and then
    # encrypts each block integer. Pass the PUBLIC key to encrypt.
    encryptedBlocks = []
    n, e = key

    for block in getBlocksFromText(message, blockSize):
        # ciphertext = plaintext ^ e mod n
        encryptedBlocks.append(pow(block, e, n))
    return encryptedBlocks


def readKeyFile(keyFilename):
    # Given the filename of a file that contains a public or private key,
    # return the key as a (n,e) or (n,d) tuple value.
    fo = open(keyFilename)
    content = fo.read()
    fo.close()
    keySize, n, EorD = content.split(',')
    return (int(keySize), int(n), int(EorD))


def encryptAndWriteToFile(messageFilename, keyFilename, message, blockSize=DEFAULT_BLOCK_SIZE):
    # Using a key from a key file, encrypt the message and save it to a
    # file. Returns the encrypted message string.
    keySize, n, e = readKeyFile(keyFilename)

    # Check that key size is greater than block size.
    if keySize < blockSize * 8: # * 8 to convert bytes to bits
        sys.exit('ERROR: Block size is %s bits and key size is %s bits. The RSA cipher requires the block size to be equal to or greater than the key size. Either decrease the block size or use different keys.' % (blockSize * 8, keySize))

    # Encrypt the message
    encryptedBlocks = encryptMessage(message, (n, e), blockSize)

    # Convert the large int values to one string value.
    for i in range(len(encryptedBlocks)):
        encryptedBlocks[i] = str(encryptedBlocks[i])
    encryptedContent = ','.join(encryptedBlocks)

    # Write out the encrypted string to the output file.
    encryptedContent = '%s_%s_%s' % (len(message), blockSize, encryptedContent)
    fo = open(messageFilename, 'w')
    fo.write(encryptedContent)
    fo.close()
    # Also return the encrypted string.
    return encryptedContent

**Question:** Explain the encryption process. Try it for different messages.

In [23]:
# Runs a test that encrypts a message to a file or decrypts a message
# from a file.
filename = 'encrypted_file.txt' # the file to write to/read from

message = '''"Journalists belong in the gutter because that is where the ruling classes throw their guilty secrets." -Gerald Priestland "The Founding Fathers gave the free press the protection it must have to bare the secrets of government and inform the people." -Hugo Black'''
pubKeyFilename = 'your_file_pubkey.txt'
print('Encrypting and writing to %s...' % (filename))
encryptedText = encryptAndWriteToFile(filename, pubKeyFilename, message)

print('Encrypted text:')
print(encryptedText)

Encrypting and writing to encrypted_file.txt...
Encrypted text:
262_128_4491404762582767725055064877046695037100495025640223247341922334754265692906982058556661088790012590635490806380963860834598286219808801722500476497107476051472772654004869893778937082211293953634983854407658775560149504146633889276470396399296785243149955562435443154798325819396091106489203101658848063149154252948499958711238442093689565957134167120618360996979451824442163460540029651396089773003280362709211454292109990191388295872516479405252980185459042564322953576687416406043807178530247325007706649905326878760171039172440681047223647263311363661418444291881438044126542711721784888163071671200015467099023,78720781447774077042571479218713787117072695674757226029981063580634692336589964757496274067224562799057928617767697450894573023943733389294799679126278706433579961461186297663939604450889970683789291495389285303468312723772372625799704372055449910932859960426648353838630584423483535840909444798309589897990254

**Question:** Instead of writing the message, propose a new function that read a file containing the plaintext (message) and save it in the message variable.

# RSA decryption

In [24]:
# IMPORTANT: The block size MUST be less than or equal to the key size!
# (Note: The block size is in bytes, the key size is in bits. There
# are 8 bits in 1 byte.)

DEFAULT_BLOCK_SIZE = 128 # 128 bytes
BYTE_SIZE = 256 # One byte has 256 different values.

def getTextFromBlocks(blockInts, messageLength, blockSize=DEFAULT_BLOCK_SIZE):
    # Converts a list of block integers to the original message string.
    # The original message length is needed to properly convert the last
    # block integer.
    message = []
    for blockInt in blockInts:
        blockMessage = []
        for i in range(blockSize - 1, -1, -1):
            if len(message) + i < messageLength:
                # Decode the message string for the 128 (or whatever
                # blockSize is set to) characters from this block integer.
                asciiNumber = blockInt // (BYTE_SIZE ** i)
                blockInt = blockInt % (BYTE_SIZE ** i)
                blockMessage.insert(0, chr(asciiNumber))
        message.extend(blockMessage)
    return ''.join(message)

def decryptMessage(encryptedBlocks, messageLength, key, blockSize=DEFAULT_BLOCK_SIZE):
    # Decrypts a list of encrypted block ints into the original message
    # string. The original message length is required to properly decrypt
    # the last block. Be sure to pass the PRIVATE key to decrypt.
    decryptedBlocks = []
    n, d = key
    for block in encryptedBlocks:
        # plaintext = ciphertext ^ d mod n
        decryptedBlocks.append(pow(block, d, n))
    return getTextFromBlocks(decryptedBlocks, messageLength, blockSize)

def readFromFileAndDecrypt(messageFilename, keyFilename):
    # Using a key from a key file, read an encrypted message from a file
    # and then decrypt it. Returns the decrypted message string.
    keySize, n, d = readKeyFile(keyFilename)


    # Read in the message length and the encrypted message from the file.
    fo = open(messageFilename)
    content = fo.read()
    messageLength, blockSize, encryptedMessage = content.split('_')
    messageLength = int(messageLength)
    blockSize = int(blockSize)

    # Check that key size is greater than block size.
    if keySize < blockSize * 8: # * 8 to convert bytes to bits
        sys.exit('ERROR: Block size is %s bits and key size is %s bits. The RSA cipher requires the block size to be equal to or greater than the key size. Did you specify the correct key file and encrypted file?' % (blockSize * 8, keySize))

    # Convert the encrypted message into large int values.
    encryptedBlocks = []
    for block in encryptedMessage.split(','):
        encryptedBlocks.append(int(block))

    # Decrypt the large int values.
    return decryptMessage(encryptedBlocks, messageLength, (n, d), blockSize)


**Question:** Explain the decryption process. Try it for different ciphertexts.

In [25]:
privKeyFilename = 'your_file_privkey.txt'
print('Reading from %s and decrypting...' % (filename))
decryptedText = readFromFileAndDecrypt(filename, privKeyFilename)

print('Decrypted text:')
print(decryptedText)

Reading from encrypted_file.txt and decrypting...
Decrypted text:
"Journalists belong in the gutter because that is where the ruling classes throw their guilty secrets." -Gerald Priestland "The Founding Fathers gave the free press the protection it must have to bare the secrets of government and inform the people." -Hugo Black


Encrypt file

In [34]:
def encryptFile(messageFilename, keyFilename, blockSize=DEFAULT_BLOCK_SIZE):
    # Using a key from a key file, encrypt the message and save it to a
    # file. Returns the encrypted message string.
    keySize, n, e = readKeyFile(keyFilename)

    # Check that key size is greater than block size.
    if keySize < blockSize * 8: # * 8 to convert bytes to bits
        sys.exit('ERROR: Block size is %s bits and key size is %s bits. The RSA cipher requires the block size to be equal to or greater than the key size. Either decrease the block size or use different keys.' % (blockSize * 8, keySize))

    # Encrypt the message
    messageLength = 0
    encryptedBlocks = []
    with open(messageFilename, "rb") as f:
        bytes = f.read()
        encryptedBlocks.append(encryptMessage(str(bytes), (n, e), blockSize))
        messageLength = len(str(bytes))
        f.close()
    # Convert the large int values to one string value.
    for i in range(len(encryptedBlocks)):
        encryptedBlocks[i] = str(encryptedBlocks[i])
    encryptedContent = ','.join(encryptedBlocks)

    # Write out the encrypted string to the output file.
    encryptedContent = '%s_%s_%s' % (messageLength, blockSize, encryptedContent.replace('[','').replace(']',''))
    fo = open(messageFilename, 'w')
    fo.write(encryptedContent)
    fo.close()
    # Also return the encrypted string.
    return encryptedContent

In [32]:
filename = 'sffl.pdf' # the file to write to/read from

pubKeyFilename = 'your_file_pubkey.txt'
print('Encrypting and writing to %s...' % (filename))
encryptedText = encryptFile(filename,pubKeyFilename)

Encrypting and writing to sffl.pdf...


decrypt

In [35]:
def decryptFile(filename,privKeyFilename):
    print('Reading from %s and decrypting...' % (filename))
    decryptedText = readFromFileAndDecrypt(filename, privKeyFilename)
    exec("open('%s','wb').write(%s)" % (filename,decryptedText))
    print('File decrypted with success')

In [38]:
filename = 'sffl.pdf'
privKeyFilename = 'your_file_privkey.txt'
decryptFile(filename,privKeyFilename)

Reading from sffl.pdf and decrypting...
File decrypted with success
